# Analysis of time series and integration

In [1]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetup
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario
import seaborn as sns
import datetime
import pickle

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
outdir = 'model_output/'
when = 'future'

n_int_steps = 5
nnodes = 107  # nodes
ndays = 90

setup = ItalySetup(nnodes, ndays, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'model_output/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)

Loaded Italy Setup with 107 nodes.


In [57]:
agestrat = pd.read_csv('italy-data/province_age_data.csv',header=1)
agestrat['Totale'] = agestrat['Totale Maschi'] + agestrat['Totale Femmine']
agestrat = agestrat[agestrat['Età'] != 'Totale']
agestrat['Category'] = ''
agestrat.loc[agestrat['Età'].astype(int) < 16, 'Category'] = '0-16'
agestrat.loc[agestrat['Età'].astype(int) > 65, 'Category'] = '65+'
agestrat.loc[agestrat['Category']== '', 'Category'] = '16-65'
agestrat = agestrat[['Category', 'Codice provincia', 'Totale', 'Provincia']]
agestrat = agestrat.groupby(['Category', 'Codice provincia', 'Provincia']).sum().reset_index().sort_values('Codice provincia').reset_index()
agestrat

,index,Category,Codice provincia,Provincia,Totale
0,0,0-16,1,Torino,309518
1,214,65+,1,Torino,539520
2,107,16-65,1,Torino,1420082
3,1,0-16,2,Vercelli,21245
4,215,65+,2,Vercelli,44121
...,...,...,...,...,...
316,212,16-65,110,Barletta-Andria-Trani,260180
317,105,0-16,110,Barletta-Andria-Trani,60723
318,106,0-16,111,Sud Sardegna,40445
319,213,16-65,111,Sud Sardegna,231197


In [59]:
geodata = pd.read_csv('italy-data/geodata.csv')

geodata = geodata.reset_index(drop=True)

In [65]:
geodata['population']

0      2259500
1       170910
2       369020
3       587100
4       214640
        ...   
102     158350
103     873940
104     173800
105     390010
106     350720
Name: population, Length: 107, dtype: int64

In [64]:
agestrat.groupby(['Codice provincia', 'Provincia']).sum()['Totale'].re

Codice provincia  Provincia            
1                 Torino                   2269120
2                 Vercelli                  172307
3                 Novara                    369595
4                 Cuneo                     588559
5                 Asti                      215884
                                            ...   
103               Verbano-Cusio-Ossola      159159
108               Monza e della Brianza     871698
109               Fermo                     174338
110               Barletta-Andria-Trani     391224
111               Sud Sardegna              353830
Name: Totale, Length: 107, dtype: int64

In [11]:
agestrat['Età'].astype(int)

0          0
1          1
2          2
3          3
4          4
        ... 
10908     96
10909     97
10910     98
10911     99
10912    100
Name: Età, Length: 10807, dtype: int64